# Import Dependencies

In [1]:
import pandas as pd
import numpy as np

# Read csv and count rows

In [2]:
#First source
athlete_event = pd.read_csv("Resource/athlete_events.csv")
#Second source
noc_regions = pd.read_csv("Resource/noc_regions.csv")
#Check for Nans
athlete_event.count()

ID        271116
Name      271116
Sex       271116
Age       261642
Height    210945
Weight    208241
Team      271116
NOC       271116
Games     271116
Year      271116
Season    271116
City      271116
Sport     271116
Event     271116
Medal      39783
dtype: int64

# Replace NaNs with No Metals for countries that got no medals

In [3]:
#replace Nan with 0
athlete_event_1 = athlete_event.replace(to_replace = np.nan, value = 0)
#replace 0 with None
athlete_event_1["Medal"]= athlete_event_1["Medal"].replace(0, "None")
athlete_event_1.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,0.0,0.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,0.0,0.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


# Merge and clean

In [4]:
merge = pd.merge(athlete_event_1, noc_regions, 
                   on='NOC', 
                   how='inner')
#delete Note and 'Games' column
merge.drop(merge.columns[16], inplace=True, axis=1)
merge.drop(merge.columns[8], inplace=True, axis=1)
merge.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,Season,City,Sport,Event,Medal,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None,China
2,602,Abudoureheman,M,22.0,182.0,75.0,China,CHN,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,None,China
3,1463,Ai Linuer,M,25.0,160.0,62.0,China,CHN,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",None,China
4,1464,Ai Yanhan,F,14.0,168.0,54.0,China,CHN,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,None,China


# Reorder Columns and rename

In [5]:
merged_2 = merge[["ID","NOC","Team", "region", "City","Name","Sex","Age","Height","Weight",
                  "Year","Season","Sport","Event","Medal"]]
#Capitalize 'region'
merged_2 = merged_2.rename(columns = {"region": "Region"})

merged_2.head()

,ID,NOC,Team,Region,City,Name,Sex,Age,Height,Weight,Year,Season,Sport,Event,Medal
0,1,CHN,China,China,Barcelona,A Dijiang,M,24.0,180.0,80.0,1992,Summer,Basketball,Basketball Men's Basketball,None
1,2,CHN,China,China,London,A Lamusi,M,23.0,170.0,60.0,2012,Summer,Judo,Judo Men's Extra-Lightweight,None
2,602,CHN,China,China,Sydney,Abudoureheman,M,22.0,182.0,75.0,2000,Summer,Boxing,Boxing Men's Middleweight,None
3,1463,CHN,China,China,Athina,Ai Linuer,M,25.0,160.0,62.0,2004,Summer,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",None
4,1464,CHN,China,China,Rio de Janeiro,Ai Yanhan,F,14.0,168.0,54.0,2016,Summer,Swimming,Swimming Women's 200 metres Freestyle,None


# Set data up for calculations

In [6]:
Cleaned = merged_2[["ID","NOC","Team", "Region", "City","Name","Sex","Age","Height","Weight",
                               "Year","Season","Sport","Event","Medal"]]

#pivot medals and make medal calculations
medals = pd.DataFrame({
    "Medals" : Cleaned["Medal"]
})

medals = pd.get_dummies(medals)
Cleaned["Bronze"] = medals["Medals_Bronze"]
Cleaned["Silver"] = medals["Medals_Silver"]
Cleaned["Gold"] = medals["Medals_Gold"]
Cleaned["No Win"] = medals["Medals_None"]
Cleaned["Attempts"] = Cleaned["Bronze"] + Cleaned["Silver"] + Cleaned["Gold"] + Cleaned["No Win"]
Cleaned["Wins"] = Cleaned["Bronze"] + Cleaned["Silver"] + Cleaned["Gold"]
Cleaned.head(-100)

,ID,NOC,Team,Region,City,Name,Sex,Age,Height,Weight,...,Season,Sport,Event,Medal,Bronze,Silver,Gold,No Win,Attempts,Wins
0,1,CHN,China,China,Barcelona,A Dijiang,M,24.0,180.0,80.0,...,Summer,Basketball,Basketball Men's Basketball,None,0,0,0,1,1,0
1,2,CHN,China,China,London,A Lamusi,M,23.0,170.0,60.0,...,Summer,Judo,Judo Men's Extra-Lightweight,None,0,0,0,1,1,0
2,602,CHN,China,China,Sydney,Abudoureheman,M,22.0,182.0,75.0,...,Summer,Boxing,Boxing Men's Middleweight,None,0,0,0,1,1,0
3,1463,CHN,China,China,Athina,Ai Linuer,M,25.0,160.0,62.0,...,Summer,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",None,0,0,0,1,1,0
4,1464,CHN,China,China,Rio de Janeiro,Ai Yanhan,F,14.0,168.0,54.0,...,Summer,Swimming,Swimming Women's 200 metres Freestyle,None,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270662,47815,MHL,Marshall Islands,Marshall Islands,London,"Ann-Marie ""Annie"" Hepler",F,16.0,167.0,68.0,...,Summer,Swimming,Swimming Women's 50 metres Freestyle,None,0,0,0,1,1,0
270663,54016,MHL,Marshall Islands,Marshall Islands,Beijing,Anju Jason,M,20.0,178.0,80.0,...,Summer,Taekwondo,Taekwondo Men's Welterweight,None,0,0,0,1,1,0
270664,60681,MHL,Marshall Islands,Marshall Islands,Beijing,Julianne Kirchner,F,16.0,138.0,54.0,...,Summer,Swimming,Swimming Women's 50 metres Freestyle,None,0,0,0,1,1,0
270665,85287,MHL,Marshall Islands,Marshall Islands,Beijing,Haley Nicole Nemra,F,18.0,168.0,59.0,...,Summer,Athletics,Athletics Women's 800 metres,None,0,0,0,1,1,0


In [7]:
Cleaned.count()

ID          270767
NOC         270767
Team        270767
Region      270746
City        270767
Name        270767
Sex         270767
Age         270767
Height      270767
Weight      270767
Year        270767
Season      270767
Sport       270767
Event       270767
Medal       270767
Bronze      270767
Silver      270767
Gold        270767
No Win      270767
Attempts    270767
Wins        270767
dtype: int64

# Check Data types and change if needed

In [154]:
Cleaned.loc[:, "Bronze"] =Cleaned.loc[:, "Bronze"].astype("int")
Cleaned.loc[:, "Silver"] =Cleaned.loc[:, "Silver"].astype("int")
Cleaned.loc[:, "Gold"] =Cleaned.loc[:, "Gold"].astype("int")
Cleaned.loc[:, "No Win"] =Cleaned.loc[:, "No Win"].astype("int")
Cleaned.loc[:, "Attempts"] =Cleaned.loc[:, "Attempts"].astype("int")
Cleaned.loc[:, "Wins"] =Cleaned.loc[:, "Wins"].astype("int")
Cleaned.loc[:, "Age"] =Cleaned.loc[:, "Age"].astype("int")
Cleaned.loc[:, "Height"] =Cleaned.loc[:, "Height"].astype("int")
Cleaned.loc[:, "Weight"] =Cleaned.loc[:, "Weight"].astype("int")
Cleaned.dtypes

ID           int64
NOC         object
Team        object
Region      object
City        object
Name        object
Sex         object
Age          int32
Height       int32
Weight       int32
Year         int64
Season      object
Sport       object
Event       object
Medal       object
Bronze       int32
Silver       int32
Gold         int32
No Win       int32
Attempts     int32
Wins         int32
dtype: object

# Export

In [155]:
Cleaned.to_csv(r'Resource\Cleaned.csv', index = True)